In [1]:
import sys
import os
import re
from multiprocessing import Pool
from itertools import repeat

from datetime import datetime
from time import sleep
import hashlib
import pandas as pd
import numpy as np

In [3]:
import bucket_manager as bm

In [4]:
# Initiate timing
start = datetime.now()
# Set the source directory, bucket name, and destination directory
source_dir = f"/rds/project/rds-rPTGgs6He74/ras81/lsst-ir-fusion/{sys.argv[1]}"
log = f"{'-'.join(source_dir.split('/')[-2:])}-files.csv"
destination_dir = f"ip005-ras81-lsst-ir-fusion/{sys.argv[1]}"
folders = []
folder_files = []
ncores = 1 # change to adjust number of CPUs (= number of concurrent connections)
perform_checksum = True
upload_checksum = False
dryrun = False

In [5]:
# Add titles to log file
with open(log, 'w') as logfile: # elsewhere open(log, 'a')
    logfile.write('LOCAL_FOLDER,LOCAL_PATH,FILE_SIZE,BUCKET_NAME,DESTINATION_KEY,CHECKSUM,CHECKSUM_SIZE,CHECKSUM_KEY\n')

In [6]:
# Setup bucket
s3_host = 'echo.stfc.ac.uk'
keys = bm.get_keys(os.sep.join([os.environ['HOME'],'lsst_keys.json']))
access_key = keys['access_key']
secret_key = keys['secret_key']

In [7]:
client = bm.get_client(access_key, secret_key, s3_host)

In [8]:
client

In [9]:
bucket_name = 'csd3-backup-test'
if dryrun:
    mybucket = 'dummy_bucket'

In [11]:
if bucket_name not in bm.bucket_list(client):
    if not dryrun:
            mybucket = client.create_bucket(bucket_name)
            print(f'Added bucket: {bucket_name}')
else:
    if not dryrun:
        print(f'Bucket exists: {bucket_name}')
        #sys.exit('Bucket exists.')
        mybucket = client.get_bucket(bucket_name)
    else:
        print(f'Bucket exists: {bucket_name}')
        print('dryrun = True, so continuing.')

TypeError: create_bucket() only accepts keyword arguments.

In [ ]:












# Process the files in parallel
print(f'Starting processing at {datetime.now()}, elapsed time = {datetime.now() - start}')
process_files(mybucket, source_dir, destination_dir, ncores, perform_checksum, log)

# Complete
print(f'Finished at {datetime.now()}, elapsed time = {datetime.now() - start}')